In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime, timedelta


In [ ]:
root_path = './dataset/'

In [ ]:
comportamiento_df = pd.read_csv(f'{root_path}comportamiento_tarjetasvisa.csv', sep = ',')

In [ ]:
informacion_df = pd.read_csv(f'{root_path}informacion_adicional_tarjetas.csv', sep = ',')

In [5]:
comportamiento_df['ID_CLIENTE'].value_counts().shape[0]

45985

### Definición de mal pagador

In [ ]:
comportamiento_df.drop_duplicates()
comportamiento_df

In [ ]:
comportamiento_df.isnull().sum()

##### 20% de las observaciones son nulas en columna DIAS_VENCIDOS

In [ ]:
grupo_df = comportamiento_df.groupby(["ID_CLIENTE","MESES"]).max()
grupo_df

In [ ]:
filtro_nulos = comportamiento_df.groupby('ID_CLIENTE').filter(lambda x:x['DIAS_VENCIDOS'].isnull().any())
filtro_nulos['ID_CLIENTE'].value_counts()

In [ ]:
comportamiento_df['ID_CLIENTE'].value_counts()

##### De un total de 45,985 clientes con comportamiento del manejo de tarjeta, 25,005 clientes tienen al menos un campo nulo en dias vencidos.

##### Se consideran todas las observaciones para definir el tipo de pagador toda vez que podemos asegurar un mal comportamiento con información cierta.

##### Score de comportamiento:
Mayor que 25 y menor igual que 30, se puntúa el comportamiento con 0.25.
Mayor que 30, se puntúa con 1.

In [ ]:
# funcion para score comportamiento
def score_comportamiento(dias_vencidos):
    if dias_vencidos > 25 and dias_vencidos <= 30:
        return 0.25
    elif dias_vencidos > 30:
        return 1
    else:
        return 0

In [ ]:
comportamiento_df['SCORE'] = comportamiento_df['DIAS_VENCIDOS'].apply(score_comportamiento)
comportamiento_df['MESES_ANTIGUEDAD'] = (comportamiento_df.groupby('ID_CLIENTE')['MESES'].transform('min')) * (-1)
comportamiento_df

In [18]:
calificacion_df = comportamiento_df[["ID_CLIENTE","MESES_ANTIGUEDAD","SCORE"]].groupby(["ID_CLIENTE","MESES_ANTIGUEDAD"]).sum().reset_index()
calificacion_df

,ID_CLIENTE,MESES_ANTIGUEDAD,SCORE
0,5001711,3,0.00
1,5001712,18,0.25
2,5001713,21,0.00
3,5001714,14,0.00
4,5001715,59,0.00
...,...,...,...
45980,5150482,28,0.25
45981,5150483,17,0.00
45982,5150484,12,0.50
45983,5150485,1,0.25


In [ ]:
# definir etiqueta de mal pagador
def tipo_cliente(meses_antiguedad, score_total):
    if meses_antiguedad <= 6 and score_total > 0:
        return 'mal pagador'
    elif meses_antiguedad > 6 and meses_antiguedad <= 12 and score_total > 0.5:
        return 'mal pagador'
    elif meses_antiguedad > 12 and meses_antiguedad <= 24 and score_total >= 1.5:
        return 'mal pagador'
    elif meses_antiguedad > 24 and meses_antiguedad <= 36 and score_total >= 2.5:
        return 'mal pagador'
    elif meses_antiguedad > 36 and meses_antiguedad <= 48 and score_total >= 3.5:
        return 'mal pagador'
    elif meses_antiguedad > 48 and meses_antiguedad <= 60 and score_total >= 4.5:
        return 'mal pagador'
    elif meses_antiguedad > 60 and score_total >= 5.5:
        return 'mal pagador'
    else:
        return 'buen pagador'

In [20]:
calificacion_df['TIPO_CLIENTE'] = calificacion_df.apply(lambda x: tipo_cliente(x['MESES_ANTIGUEDAD'], x['SCORE']), axis=1)
calificacion_df['TIPO_CLIENTE'].value_counts(normalize=True)

TIPO_CLIENTE
buen pagador    0.907426
mal pagador     0.092574
Name: proportion, dtype: float64

### Join con información adicional del cliente

In [22]:
calificacion_df.isnull().sum()

ID_CLIENTE          0
MESES_ANTIGUEDAD    0
SCORE               0
TIPO_CLIENTE        0
dtype: int64

In [ ]:
calificacion_df.info()

No se tiene valores nulos con el nuevo dataset con la definición de mal pagador

In [25]:
#Se crea un nuevo data con las dos columnas solicitadas CLIENTE_ID (ID_CLIENTE) y mal_pagador (TIPO_CLIENTE)
definicion_df = calificacion_df[["ID_CLIENTE", "TIPO_CLIENTE"]]
definicion_df

,ID_CLIENTE,TIPO_CLIENTE
0,5001711,buen pagador
1,5001712,buen pagador
2,5001713,buen pagador
3,5001714,buen pagador
4,5001715,buen pagador
...,...,...
45980,5150482,buen pagador
45981,5150483,buen pagador
45982,5150484,buen pagador
45983,5150485,mal pagador


In [ ]:
#Análisis del dataset con la información de los clientes
informacion_df.info()

In [ ]:
informacion_df.drop_duplicates()
informacion_df

,ID_CLIENTE,GENERO,TIENE_CARRO,TIENE_PROPIEDADES,N_NINOS,INGRESO_ANUAL,CATEGORIA_INGRESO,NIVEL_EDUCACION,ESTADO_CIVIL,TIPO_CASA,DIAS_DESDE_NACIMIENTO,DIAS_TRABAJANDO,TIENE_CELULAR,TIENE_NUMEROTELEF_LABORAL,TIENE_NUMEROTELEF_PERSONAL,TIENE_EMAIL,PROFESION,N_MIEMBROSFAMILIA
0,5008804,M,1,1,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
1,5008805,M,1,1,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
2,5008806,M,1,1,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
3,5008808,F,0,1,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
4,5008809,F,0,1,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438552,6840104,M,0,1,0,135000.0,Pensioner,Secondary / secondary special,Separated,House / apartment,-22717,365243,1,0,0,0,NaN,1.0
438553,6840222,F,0,0,0,103500.0,Working,Secondary / secondary special,Single / not married,House / apartment,-15939,-3007,1,0,0,0,Laborers,1.0
438554,6841878,F,0,0,0,54000.0,Commercial associate,Higher education,Single / not married,With parents,-8169,-372,1,1,0,0,Sales staff,1.0
438555,6842765,F,0,1,0,72000.0,Pensioner,Secondary / secondary special,Married,House / apartment,-21673,365243,1,0,0,0,NaN,2.0


In [ ]:
informacion_df.isnull().sum()

##### El 31% de las observaciones para la información de las tarjetas es nula en la descripción de la profesión

In [30]:
id_informacion_df = informacion_df["ID_CLIENTE"].value_counts()
id_informacion_df

ID_CLIENTE
7137299    2
7702238    2
7282535    2
7243768    2
7050948    2
          ..
5690727    1
6621262    1
6621261    1
6621260    1
6842885    1
Name: count, Length: 438510, dtype: int64

In [31]:
#Observando los valores repetidos
repetido_informacion_df = id_informacion_df[id_informacion_df > 1].index
repetido_informacion_df

Index([7137299, 7702238, 7282535, 7243768, 7050948, 7602432, 7036518, 7045885,
       7618285, 7089090, 7052783, 7416167, 7023108, 7045794, 7053557, 7207977,
       7836711, 7772847, 7154819, 7052812, 7099881, 7022327, 7833087, 7743418,
       7836971, 7372589, 7636756, 7636389, 7213374, 7317997, 7838075, 7023651,
       7155150, 7091721, 7046068, 7742298, 7744386, 7702516, 7022197, 7154598,
       7742853, 7090931, 7135270, 7024111, 7174719, 7603224, 7576316],
      dtype='int64', name='ID_CLIENTE')

In [32]:
repetido_filtro_df = informacion_df[informacion_df['ID_CLIENTE'].isin(repetido_informacion_df)].sort_values(by="ID_CLIENTE")
repetido_filtro_df

,ID_CLIENTE,GENERO,TIENE_CARRO,TIENE_PROPIEDADES,N_NINOS,INGRESO_ANUAL,CATEGORIA_INGRESO,NIVEL_EDUCACION,ESTADO_CIVIL,TIPO_CASA,DIAS_DESDE_NACIMIENTO,DIAS_TRABAJANDO,TIENE_CELULAR,TIENE_NUMEROTELEF_LABORAL,TIENE_NUMEROTELEF_PERSONAL,TIENE_EMAIL,PROFESION,N_MIEMBROSFAMILIA
426818,7022197,M,1,1,3,135000.0,Working,Secondary / secondary special,Married,House / apartment,-11945,-735,1,0,0,1,Laborers,5.0
425023,7022197,F,0,1,0,450000.0,Commercial associate,Higher education,Separated,House / apartment,-19813,-1799,1,0,0,1,NaN,1.0
431545,7022327,F,0,1,0,135000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-14771,-5298,1,0,0,0,High skill tech staff,1.0
431911,7022327,M,1,1,0,256500.0,Commercial associate,Higher education,Married,House / apartment,-21503,-1674,1,0,0,1,Core staff,2.0
425486,7023108,M,1,1,1,67500.0,Working,Secondary / secondary special,Married,House / apartment,-15156,-1696,1,1,0,0,Core staff,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426563,7836711,F,0,1,2,292500.0,Working,Higher education,Married,House / apartment,-13747,-4386,1,0,1,0,Accountants,4.0
421464,7836971,M,1,0,1,157500.0,Working,Secondary / secondary special,Married,House / apartment,-13771,-5520,1,0,0,0,NaN,3.0
428620,7836971,F,0,1,0,103500.0,Working,Secondary / secondary special,Civil marriage,House / apartment,-13383,-2798,1,0,1,0,Sales staff,2.0
422068,7838075,M,0,1,0,337500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-18198,-1275,1,0,0,1,Drivers,2.0


Se descartan los ids que tienen más de una información porque los valores cambiantes no siguen ningún patrón (MCAR)

In [34]:
#Obteniedo los id con información única
filtered_informacion_df = id_informacion_df[id_informacion_df == 1].index
filtered_informacion_df

Index([6147693, 6147702, 6147689, 6147705, 6147690, 6147691, 6147704, 6147703,
       6147634, 6147638,
       ...
       6621267, 6621266, 6621265, 6621264, 6621263, 5690727, 6621262, 6621261,
       6621260, 6842885],
      dtype='int64', name='ID_CLIENTE', length=438463)

In [35]:
#Filtrando los ids únicos
unicos_informacion_df = informacion_df[informacion_df['ID_CLIENTE'].isin(filtered_informacion_df)]
unicos_informacion_df

,ID_CLIENTE,GENERO,TIENE_CARRO,TIENE_PROPIEDADES,N_NINOS,INGRESO_ANUAL,CATEGORIA_INGRESO,NIVEL_EDUCACION,ESTADO_CIVIL,TIPO_CASA,DIAS_DESDE_NACIMIENTO,DIAS_TRABAJANDO,TIENE_CELULAR,TIENE_NUMEROTELEF_LABORAL,TIENE_NUMEROTELEF_PERSONAL,TIENE_EMAIL,PROFESION,N_MIEMBROSFAMILIA
0,5008804,M,1,1,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
1,5008805,M,1,1,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
2,5008806,M,1,1,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
3,5008808,F,0,1,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
4,5008809,F,0,1,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438552,6840104,M,0,1,0,135000.0,Pensioner,Secondary / secondary special,Separated,House / apartment,-22717,365243,1,0,0,0,NaN,1.0
438553,6840222,F,0,0,0,103500.0,Working,Secondary / secondary special,Single / not married,House / apartment,-15939,-3007,1,0,0,0,Laborers,1.0
438554,6841878,F,0,0,0,54000.0,Commercial associate,Higher education,Single / not married,With parents,-8169,-372,1,1,0,0,Sales staff,1.0
438555,6842765,F,0,1,0,72000.0,Pensioner,Secondary / secondary special,Married,House / apartment,-21673,365243,1,0,0,0,NaN,2.0


94 registros se han descartado de la información de clientes

In [37]:
unificado_df = unicos_informacion_df.merge(definicion_df, how="left", on="ID_CLIENTE")
unificado_df

,ID_CLIENTE,GENERO,TIENE_CARRO,TIENE_PROPIEDADES,N_NINOS,INGRESO_ANUAL,CATEGORIA_INGRESO,NIVEL_EDUCACION,ESTADO_CIVIL,TIPO_CASA,DIAS_DESDE_NACIMIENTO,DIAS_TRABAJANDO,TIENE_CELULAR,TIENE_NUMEROTELEF_LABORAL,TIENE_NUMEROTELEF_PERSONAL,TIENE_EMAIL,PROFESION,N_MIEMBROSFAMILIA,TIPO_CLIENTE
0,5008804,M,1,1,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,buen pagador
1,5008805,M,1,1,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,buen pagador
2,5008806,M,1,1,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0,buen pagador
3,5008808,F,0,1,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,buen pagador
4,5008809,F,0,1,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,buen pagador
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438458,6840104,M,0,1,0,135000.0,Pensioner,Secondary / secondary special,Separated,House / apartment,-22717,365243,1,0,0,0,NaN,1.0,NaN
438459,6840222,F,0,0,0,103500.0,Working,Secondary / secondary special,Single / not married,House / apartment,-15939,-3007,1,0,0,0,Laborers,1.0,NaN
438460,6841878,F,0,0,0,54000.0,Commercial associate,Higher education,Single / not married,With parents,-8169,-372,1,1,0,0,Sales staff,1.0,NaN
438461,6842765,F,0,1,0,72000.0,Pensioner,Secondary / secondary special,Married,House / apartment,-21673,365243,1,0,0,0,NaN,2.0,NaN


In [ ]:
#Analizando la información unificada
unificado_df.info()

Hay 402006 registros lo que representa el 91 % de información nula porque los ids de clientes no estaban en la definición del tipo de cliente.
Es un categoría necesaria para analizar el tipo de cliente se va a trabajar con clientes que si estén categorizados

In [40]:
unificado_df[unificado_df.duplicated() == True]

,ID_CLIENTE,GENERO,TIENE_CARRO,TIENE_PROPIEDADES,N_NINOS,INGRESO_ANUAL,CATEGORIA_INGRESO,NIVEL_EDUCACION,ESTADO_CIVIL,TIPO_CASA,DIAS_DESDE_NACIMIENTO,DIAS_TRABAJANDO,TIENE_CELULAR,TIENE_NUMEROTELEF_LABORAL,TIENE_NUMEROTELEF_PERSONAL,TIENE_EMAIL,PROFESION,N_MIEMBROSFAMILIA,TIPO_CLIENTE


No se tiene valores duplicados ya que desde el inicio se usó la función drop_duplicates para no trabajar con valores repetidos

In [ ]:
#Se conservan los registros que no tienen valores nulos para el tipo de cliente según la definición de mal pagador
clientes_df = unificado_df[unificado_df["TIPO_CLIENTE"].isnull() == False]
clientes_df.info()

In [43]:
clientes_df[["TIPO_CLIENTE", "ID_CLIENTE"]].groupby("TIPO_CLIENTE").count()

,ID_CLIENTE
TIPO_CLIENTE,
buen pagador,32959
mal pagador,3498


### Data quality

El 31% de la información que se ha recolectado del cliente no ha indicado su profesión.

In [46]:
segun_genero_df = clientes_df[["GENERO", "PROFESION", "ID_CLIENTE"]].groupby("GENERO").count()
segun_genero_df

,PROFESION,ID_CLIENTE
GENERO,,
F,15630,24430
M,9504,12027


In [47]:
segun_genero_df["SIN PROFESION"] = segun_genero_df["ID_CLIENTE"] - segun_genero_df["PROFESION"]
segun_genero_df["% SIN PROFESION"] = (segun_genero_df["SIN PROFESION"] * 100)/segun_genero_df["ID_CLIENTE"]
segun_genero_df

,PROFESION,ID_CLIENTE,SIN PROFESION,% SIN PROFESION
GENERO,,,,
F,15630,24430,8800,36.021285
M,9504,12027,2523,20.977800


El 36 % de mujeres no ha indicado que profesión tienen y el 21% de los hombres prefirieron no decir a que se dedican

### Feature transformation

In [50]:
clientes_df.describe()

,ID_CLIENTE,TIENE_CARRO,TIENE_PROPIEDADES,N_NINOS,INGRESO_ANUAL,DIAS_DESDE_NACIMIENTO,DIAS_TRABAJANDO,TIENE_CELULAR,TIENE_NUMEROTELEF_LABORAL,TIENE_NUMEROTELEF_PERSONAL,TIENE_EMAIL,N_MIEMBROSFAMILIA
count,3.645700e+04,36457.000000,36457.000000,36457.000000,3.645700e+04,36457.000000,36457.000000,36457.0,36457.000000,36457.000000,36457.000000,36457.000000
mean,5.078227e+06,0.379708,0.672189,0.430315,1.866857e+05,-15975.173382,59262.935568,1.0,0.225526,0.294813,0.089722,2.198453
std,4.187524e+04,0.485321,0.469422,0.742367,1.017892e+05,4200.549944,137651.334859,0.0,0.417934,0.455965,0.285787,0.911686
min,5.008804e+06,0.000000,0.000000,0.000000,2.700000e+04,-25152.000000,-15713.000000,1.0,0.000000,0.000000,0.000000,1.000000
25%,5.042028e+06,0.000000,0.000000,0.000000,1.215000e+05,-19438.000000,-3153.000000,1.0,0.000000,0.000000,0.000000,2.000000
50%,5.074614e+06,0.000000,1.000000,0.000000,1.575000e+05,-15563.000000,-1552.000000,1.0,0.000000,0.000000,0.000000,2.000000
75%,5.115396e+06,1.000000,1.000000,1.000000,2.250000e+05,-12462.000000,-408.000000,1.0,0.000000,1.000000,0.000000,3.000000
max,5.150487e+06,1.000000,1.000000,19.000000,1.575000e+06,-7489.000000,365243.000000,1.0,1.000000,1.000000,1.000000,20.000000


In [51]:
fecha_fijada = datetime(2024, 6, 29).date()
clientes_df['FECHA_NACIMIENTO'] = clientes_df['DIAS_DESDE_NACIMIENTO'].apply(lambda x: fecha_fijada + timedelta(days=x))
clientes_df['ANIO_NACIMIENTO'] = clientes_df['FECHA_NACIMIENTO'].apply(lambda x: x.year)
clientes_df

C:\Users\mrumb\AppData\Local\Temp\ipykernel_20616\3620495674.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clientes_df['FECHA_NACIMIENTO'] = clientes_df['DIAS_DESDE_NACIMIENTO'].apply(lambda x: fecha_fijada + timedelta(days=x))
C:\Users\mrumb\AppData\Local\Temp\ipykernel_20616\3620495674.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clientes_df['ANIO_NACIMIENTO'] = clientes_df['FECHA_NACIMIENTO'].apply(lambda x: x.year)


,ID_CLIENTE,GENERO,TIENE_CARRO,TIENE_PROPIEDADES,N_NINOS,INGRESO_ANUAL,CATEGORIA_INGRESO,NIVEL_EDUCACION,ESTADO_CIVIL,TIPO_CASA,...,DIAS_TRABAJANDO,TIENE_CELULAR,TIENE_NUMEROTELEF_LABORAL,TIENE_NUMEROTELEF_PERSONAL,TIENE_EMAIL,PROFESION,N_MIEMBROSFAMILIA,TIPO_CLIENTE,FECHA_NACIMIENTO,ANIO_NACIMIENTO
0,5008804,M,1,1,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,...,-4542,1,1,0,0,NaN,2.0,buen pagador,1991-08-17,1991
1,5008805,M,1,1,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,...,-4542,1,1,0,0,NaN,2.0,buen pagador,1991-08-17,1991
2,5008806,M,1,1,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,...,-1134,1,0,0,0,Security staff,2.0,buen pagador,1965-09-13,1965
3,5008808,F,0,1,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,...,-3051,1,0,1,1,Sales staff,1.0,buen pagador,1972-03-04,1972
4,5008809,F,0,1,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,...,-3051,1,0,1,1,Sales staff,1.0,buen pagador,1972-03-04,1972
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
434714,5149828,M,1,1,0,315000.0,Working,Secondary / secondary special,Married,House / apartment,...,-2420,1,0,0,0,Managers,2.0,mal pagador,1976-12-30,1976
434715,5149834,F,0,1,0,157500.0,Commercial associate,Higher education,Married,House / apartment,...,-1325,1,0,1,1,Medicine staff,2.0,mal pagador,1990-07-31,1990
434716,5149838,F,0,1,0,157500.0,Pensioner,Higher education,Married,House / apartment,...,-1325,1,0,1,1,Medicine staff,2.0,mal pagador,1990-07-31,1990
434717,5150049,F,0,1,0,283500.0,Working,Secondary / secondary special,Married,House / apartment,...,-655,1,0,0,0,Sales staff,2.0,mal pagador,1975-04-30,1975


In [52]:
#funcion para etiqueta generacional
def generacion(anio_nacimiento):
    if anio_nacimiento >= 1997:
        return 'Generación Z'
    elif 1981 <= anio_nacimiento < 1997:
        return 'Millennials'
    elif 1965 <= anio_nacimiento < 1981:
        return 'Generación X'
    elif 1946 <= anio_nacimiento < 1965:
        return 'Baby Boomers'
    else:
        return 'Generación Silenciosa'
clientes_df['GENERACION'] = clientes_df['ANIO_NACIMIENTO'].apply(generacion)

C:\Users\mrumb\AppData\Local\Temp\ipykernel_20616\2287765963.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clientes_df['GENERACION'] = clientes_df['ANIO_NACIMIENTO'].apply(generacion)


In [53]:
clientes_df['GENERACION'].value_counts()

GENERACION
Millennials     16690
Generación X    13209
Baby Boomers     4233
Generación Z     2325
Name: count, dtype: int64

### Análisis descriptivo

In [ ]:
#Descripción general de todas las variables
clientes_df.describe()

¿Cuál es la profesión de más o menos riesgo? ¿Los ingresos según la profesión tienen un impacto en el tipo de cliente?

Medidas de tendencia central

In [ ]:
clientes_df["PROFESION"].describe()

In [ ]:
proporcion_profesion = clientes_df['PROFESION'].value_counts(normalize=True)
proporcion_profesion

In [ ]:
tabla_contingencia = pd.crosstab(clientes_df['PROFESION'], clientes_df['TIPO_CLIENTE'], normalize=True)
tabla_ordenada_filas = tabla_contingencia.sort_values(by='buen pagador', ascending=False)
tabla_ordenada_filas

In [ ]:
# Gráfico de barras para la columna 'Profesion'
sns.countplot(x='PROFESION', data=clientes_df)
plt.title('Frecuencia de Profesion')
plt.xlabel('Profesión')
plt.ylabel('Frecuencia')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(x='PROFESION', hue='TIPO_CLIENTE', data=clientes_df)
plt.title('Distribución de profesión según el tipo de cliente')
plt.xlabel('Profesión')
plt.ylabel('Frecuencia')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
tabla_contingencia.plot(kind='bar', stacked=True)
plt.title('Distribución de profesión según el tipo de cliente')
plt.xlabel('Profesión')
plt.ylabel('Frecuencia')
plt.legend(title='Tipo de cliente')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

In [ ]:
clientes_df["INGRESO_MENSUAL"] = clientes_df['INGRESO_ANUAL']/12

In [ ]:
tabla_contingencia_pit = pd.crosstab([clientes_df['PROFESION'], clientes_df['TIPO_CLIENTE']], clientes_df['INGRESO_MENSUAL'], normalize=True)
tabla_contingencia_pit

In [ ]:
tabla_contingencia_pit_c = pd.crosstab([clientes_df['PROFESION'], clientes_df['INGRESO_MENSUAL']], clientes_df['TIPO_CLIENTE'], normalize=True)
tabla_contingencia_pit_c

¿El estado civil y el número de hijos de un cliente son relevantes para definir al cliente?

In [ ]:
clientes_df["ESTADO_CIVIL"].describe()

In [ ]:
clientes_df["N_NINOS"].describe()

In [ ]:
proporcion_estado_civil = clientes_df['ESTADO_CIVIL'].value_counts(normalize=True)
proporcion_estado_civil

In [ ]:
proporcion_n_ninos = clientes_df['N_NINOS'].value_counts(normalize=True)
proporcion_n_ninos

In [ ]:
tabla_contingencia_ec = pd.crosstab(clientes_df['ESTADO_CIVIL'], clientes_df['N_NINOS'])
tabla_contingencia_ec

In [ ]:
tabla_contingencia_ect = pd.crosstab(clientes_df['ESTADO_CIVIL'], clientes_df['TIPO_CLIENTE'])
tabla_ordenada_filas_ect = tabla_contingencia_ect.sort_values(by='buen pagador', ascending=False)
tabla_ordenada_filas_ect

In [ ]:
tabla_contingencia_c = pd.crosstab([clientes_df['ESTADO_CIVIL'], clientes_df['N_NINOS']], clientes_df['TIPO_CLIENTE'], normalize=True)
tabla_ordenada_filas_c = tabla_contingencia_c.sort_values(by='buen pagador', ascending=False)
tabla_ordenada_filas_c

In [ ]:
# Gráfico de barras apiladas
tabla_contingencia_c.plot(kind='bar', stacked=True)
plt.title('Distribución del tipo de cliente según el estado civil por el número de niños')
plt.xlabel('Estado civil y niños')
plt.ylabel('Frecuencia')
plt.legend(title='Tipo de cliente')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Personalizar el gráfico
# Crear el gráfico de mosaico
plt.figure(figsize=(10, 6))
mosaic(clientes_df, ['ESTADO_CIVIL', 'TIPO_CLIENTE', 'N_NINOS'], title='Gráfico de Mosaico de Estado civil, ninos y tipo cliente')
plt.show()

Distribucion y visualización

In [ ]:
clientes_df["TIENE_CARRO"].hist(bins=10, color=colors)
plt.title("Histograma de carro")
plt.xlabel("carro")
plt.ylabel("Frecuencia")
plt.show()

In [ ]:
clientes_df['TIENE_CARRO'].plot(kind='box', color=colors)
plt.title('Boxplot de carro')
plt.show()

In [ ]:
generacion_df = pd.crosstab(clientes_df['GENERACION'], clientes_df["TIPO_CLIENTE"], normalize="index")

In [56]:
generacion_df.reindex(["Generación Z", "Millennials", "Generación X", "Baby Boomers"])

TIPO_CLIENTE,buen pagador,mal pagador
GENERACION,,
Generación Z,0.872688,0.127312
Millennials,0.902516,0.097484
Generación X,0.909380,0.090620
Baby Boomers,0.910702,0.089298


### Análisis correlacional 